In [ ]:


import dash
import dash_core_components as dcc
import dash_html_components as html
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import time

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob
import time

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

consumer_key = 'sKjYALK3ugOOIDdCKjCZF33ux'
consumer_secret = 'iBlFYiZGdKtcqPd9V2O4jaQydOhiyToAmt34NdE798KxxT28Kn'
access_token = '2668936804-K0k0eP8cbrnayj6ucM8sJiTZiX1ndu9WeNr16G9'
access_token_secret = 'sObwxVsU3vTqD68g6gAdmEI1OQcnBHOmwkFkM1hD0ipSx'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

     Parameters
     ----------
    txt : string
     A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

search_term = "#Bitcoin -filter:retweets"

tweets = tw.Cursor(api.search,
                    q=search_term,
                    lang="en",
                    since='2021-01-01').items(1000)

    # Remove URLs
tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]

sentiment_objects = [TextBlob(tweet) for tweet in tweets_no_urls]

sentiment_objects[0].polarity, sentiment_objects[0]
    
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]

sentiment_values[0]

sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])

sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1],
             color="purple")

plt.title("Bitcoin Sentiment")
    

In [16]:
data = sentiment_df

app = dash.Dash(__name__)

app.layout = html.Div(
    children=[
        html.H1(children="Bitcoin Twitter Sentiment",),
        html.P(children="Current Twitter Sentiment",),
        dcc.Graph(
            figure={
                "data": [
                    {
                        "x": data["polarity"],
                        "type": "histogram",
                    },
                ],
                "layout": {"title": "Bitcoin Polarity Scores"},
            },
        ),
    ]
)

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
#Comments regarding the above code: 

#The Basis for this code was found in RealPython tutorial at realpython.com
#Created a Twitter Developer account which allowed us to pull tweets in real time 
#The initial goal was to have this run on a loop, but the data was not very helpful if we pulled on short intervals.
#A tool like this I think would only be helpful if we pulled more tweets and gauged sentiment over a longer period, i.e., Days vs. Seconds/Minutes
#As it stands now, this can be used as a simple sentiment gauge people may use to assist with trading decisions or conducting analysis. 
#When opened in Google Colab an initial "pip install" is required for DASH